# D - トレジャーハント

networkxの練習のために挑戦。[AtCoderで使えるnetworkx ケーススタディ](https://qiita.com/yH3PO4/items/ffd81081c254895939c0)を参考にした。

In [1]:
# networkxを用いない実装
from collections import defaultdict
from heapq import heappop, heappush


def dijkstra(edges, start):
    graph = defaultdict(lambda: [])
    for u, v, w in edges:
        graph[u].append((v, w))

    dist = defaultdict(lambda: float("inf"))
    dist[start] = 0
    heap = [(0, start)]

    # heapはlistなので、__len__で評価される
    while heap:
        # 現在地を更新
        d, u = heappop(heap)
        if d > dist[u]:
            continue
        # 周辺を見渡す
        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                heappush(heap, (dist[v], v))

    return dist


def treasure_hunt():
    _, m, t = map(int, input().split())
    vs = list(map(int, input().split()))
    edges = [tuple(map(int, input().split())) for _ in range(m)]

    # 経路の中で最も得点の高い頂点に、移動時間以外ずっといれば良い
    # したがって、全ての頂点に対して最短閉路を求め、それを用いて得点を算出する
    START = 1
    forward_dists = dijkstra(edges, START)
    reversed_edges = [(v, u, w) for u, v, w in edges]
    backward_dists = dijkstra(reversed_edges, START)

    max_score = 0
    for i, w in enumerate(vs):
        travel = forward_dists[i + 1] + backward_dists[i + 1]
        if travel <= t:
            score = w * (t - travel)
            max_score = max(max_score, score)

    return max_score

In [2]:
from unittest.mock import patch

with patch("builtins.input", side_effect=["2 2 5", "1 3", "1 2 2", "2 1 1"]):
    expected = 6
    actual = treasure_hunt()
    assert expected == actual, print(f"{expected=}, {actual=}")

In [3]:
from unittest.mock import patch

with patch(
    "builtins.input",
    side_effect="""
2 2 3
1 3
1 2 2
2 1 1""".strip().splitlines(),
):
    expected = 3
    actual = treasure_hunt()
    assert expected == actual, print(f"{expected=}, {actual=}")

In [4]:
from unittest.mock import patch

with patch(
    "builtins.input",
    side_effect="""
8 15 120
1 2 6 16 1 3 11 9
1 8 1
7 3 14
8 2 13
3 5 4
5 7 5
6 4 1
6 8 17
7 8 5
1 4 2
4 7 1
6 1 3
3 1 10
2 6 5
2 4 12
5 1 30
""".strip().splitlines(),
):
    expected = 1488
    actual = treasure_hunt()
    assert expected == actual, print(f"{expected=}, {actual=}")

In [8]:
# networkxを用いる実装
import networkx as nx


def treasure_hunt_x():
    n, m, t = map(int, input().split())
    vs = list(map(int, input().split()))
    # add_edges_fromはtupleではなくmapを受け取っても機能する
    edges = [map(int, input().split()) for _ in range(m)]

    graph = nx.DiGraph()
    graph.add_nodes_from(range(1, n + 1))
    graph.add_weighted_edges_from(edges)

    forward_dists = nx.single_source_dijkstra_path_length(graph, 1)
    backward_dists = nx.single_source_dijkstra_path_length(graph.reverse(copy=False), 1)

    max_score = 0
    for i, w in enumerate(vs):
        travel = forward_dists[i + 1] + backward_dists[i + 1]
        if travel <= t:
            score = w * (t - travel)
            max_score = max(max_score, score)

    return max_score

In [9]:
from unittest.mock import patch

with patch("builtins.input", side_effect=["2 2 5", "1 3", "1 2 2", "2 1 1"]):
    expected = 6
    actual = treasure_hunt_x()
    assert expected == actual, print(f"{expected=}, {actual=}")

In [10]:
from unittest.mock import patch

with patch(
    "builtins.input",
    side_effect="""
2 2 3
1 3
1 2 2
2 1 1""".strip().splitlines(),
):
    expected = 3
    actual = treasure_hunt_x()
    assert expected == actual, print(f"{expected=}, {actual=}")

In [11]:
from unittest.mock import patch

with patch(
    "builtins.input",
    side_effect="""
8 15 120
1 2 6 16 1 3 11 9
1 8 1
7 3 14
8 2 13
3 5 4
5 7 5
6 4 1
6 8 17
7 8 5
1 4 2
4 7 1
6 1 3
3 1 10
2 6 5
2 4 12
5 1 30
""".strip().splitlines(),
):
    expected = 1488
    actual = treasure_hunt_x()
    assert expected == actual, print(f"{expected=}, {actual=}")